# Final model - preprocessing and classification for NEWS

In [115]:
import pandas as pd
import numpy as np
import re
import string
import unicodedata

from transformers import pipeline
from nltk.tokenize import word_tokenize
from googletrans import Translator

#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', None)

In [79]:
import httpx

In [80]:
httpx.Timeout(5)

Timeout(timeout=5)

In [4]:
publico_path = '../extra/notícias/news_sample_1to1000_updated.csv'
observador_path = '../extra/notícias/noticias_observador_fev_a_julho.csv'

In [5]:
df_publico = pd.read_csv(publico_path, index_col='Unnamed: 0')
df_observador = pd.read_csv(observador_path, index_col='Unnamed: 0')

## Cleaning

In [6]:
# Removendo "tipo" quizz
df_publico = df_publico[df_publico["tipo"]!="QUIZZ"]

In [10]:
# colunas essenciais
df_publico = df_publico[["data", "titulo", "descricao"]]
df_observador = df_observador[["Data", "Titulo", "Desc"]]

In [17]:
# rename de colunas do observador para serem iguais às do público
df_observador.rename(columns={"Data": "data",
                              "Titulo": "titulo",
                              "Desc": "descricao"}, inplace=True)

In [18]:
df_publico.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9668 entries, 0 to 9669
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   data       9668 non-null   object
 1   titulo     9668 non-null   object
 2   descricao  9287 non-null   object
dtypes: object(3)
memory usage: 302.1+ KB


In [19]:
df_observador.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28735 entries, 0 to 28734
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   data       28735 non-null  object
 1   titulo     28735 non-null  object
 2   descricao  28735 non-null  object
dtypes: object(3)
memory usage: 898.0+ KB


In [20]:
# change data to date type
df_publico["data"] = pd.to_datetime(df_publico["data"]).dt.normalize()
df_observador["data"] = pd.to_datetime(df_observador["data"]).dt.normalize()

In [24]:
# lets consider dates between 01/01/2020 and 30/06/2020
start_date = "2020-01-01"
end_date = "2020-06-30"

df_publico = df_publico.loc[(df_publico["data"] >= start_date) & 
                            (df_publico["data"] <= end_date)]
df_observador = df_observador.loc[(df_observador["data"] >= start_date) & 
                            (df_observador["data"] <= end_date)]

In [25]:
df_publico.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8858 entries, 810 to 9669
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   data       8858 non-null   datetime64[ns]
 1   titulo     8858 non-null   object        
 2   descricao  8506 non-null   object        
dtypes: datetime64[ns](1), object(2)
memory usage: 276.8+ KB


In [26]:
df_observador.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28735 entries, 0 to 28734
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   data       28735 non-null  datetime64[ns]
 1   titulo     28735 non-null  object        
 2   descricao  28735 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 898.0+ KB


In [32]:
df_publico['source'] = 'publico'
df_observador['source'] = 'observador'

<ipython-input-32-6534b548169f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_publico['source'] = 'publico'


In [33]:
# concatenate both dataframes
df_news = pd.concat([df_publico, df_observador], ignore_index=True)

In [34]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37593 entries, 0 to 37592
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   data       37593 non-null  datetime64[ns]
 1   titulo     37593 non-null  object        
 2   descricao  37241 non-null  object        
 3   source     37593 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.1+ MB


In [35]:
df_news.head()

,data,titulo,descricao,source
0,2020-06-30,Ex-ministro da Saúde Correia de Campos alerta ...,"“Preparemo-nos para o que aí vem”, avisou o an...",publico
1,2020-06-30,Estados Unidos compram quase todo o <i>stock</...,A compra das mais de 500 mil doses do medicame...,publico
2,2020-06-30,Governo admite quarentena para hotéis com cont...,Hóspedes e funcionários também poderão ficar d...,publico
3,2020-06-30,Covid-19: quem quer ajudar os cientistas a seg...,A perda de olfacto é um dos sintomas comuns as...,publico
4,2020-06-30,"Profissionais do SNS “não deram o litro, deram...","O documento foi apresentado no Infarmed, em Li...",publico


## Preprocessing

In [144]:
stopwords = {"coronavirus","covid19", "covid19pt","covid","CoronaVirus","FiqueEmCasa", "fiqueemcasa", "Covid19","StayAtHome",
            "Coronavirus.","COVID2019", "COVID19" "Coronavirus", "STOPCOVID19", "ficaemcasa"}

# Function to clean Portuguese text and tokenize it
def preProcessing(text):
    if isinstance(text, str):
        #text = text.lower() # convert text to lower-case
        text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
        text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
        text = re.sub('RT[\s]+', '', text) # Removing RT
        text = re.sub(r'#([^\s]+)', r'\1', text) # remove the # in #hashtag
        text = re.sub(r'<.*?>', '', text) # remove HTML formatting
        text = word_tokenize(text) # remove repeated characters
        word_list = [word for word in text if word not in stopwords]
        text = ' '.join(word_list)
    else:
        text = ''
    return text

translator = Translator(timeout=httpx.Timeout(5))

# Function to translate the current text to english
def Translation(text):
    if  translator.detect(text) != 'en':
        traducao = translator.translate(text, dest='en')
        return traducao.text
    else:
        return text

In [44]:
df_news.describe()

<ipython-input-44-8670de5afc49>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_news.describe()


,data,titulo,descricao,source,titulo_clean
count,37593,37593,37241,37593,37593
unique,162,21333,20778,2,21318
top,2020-03-26 00:00:00,O que vai acontecer aos portugueses de ...,As notícias no Observador. Aconteça o que acon...,observador,"Wuhan Coronavírus : afinal , qual o risco ?"
freq,465,31,192,28735,31
first,2020-01-21 00:00:00,NaN,NaN,NaN,NaN
last,2020-06-30 00:00:00,NaN,NaN,NaN,NaN


In [47]:
df_news['titulo_clean'] = df_news['titulo'].apply(preProcessing)

In [145]:
df_news['descricao_clean'] = df_news['descricao'].apply(preProcessing)

In [83]:
titles = list(df_news["titulo_clean"].to_numpy())

translations = translator.translate(titles, dest='en')

In [146]:
desc = list(df_news['descricao_clean'].to_numpy())

translations = translator.translate(desc, dest='en')

In [152]:
translations_list = []

for translation in translations:
    translations_list.append(translation.text)

In [153]:
df_news['descricao_en'] = translations_list

In [154]:
df_news[['descricao', 'descricao_en']].head(10)

,descricao,descricao_en
0,"“Preparemo-nos para o que aí vem”, avisou o an...","“Let us prepare for what is coming”, warned th..."
1,A compra das mais de 500 mil doses do medicame...,The purchase of more than 500 thousand doses o...
2,Hóspedes e funcionários também poderão ficar d...,Guests and staff may also be quarantined in th...
3,A perda de olfacto é um dos sintomas comuns as...,Loss of smell is one of the common symptoms as...
4,"O documento foi apresentado no Infarmed, em Li...","The document was presented at Infarmed, in Lis..."
5,Joe Biden apresentou um plano para combater a ...,Joe Biden presented a plan to combat covid-19 ...
6,Proposta do PSD prevê que inquilinos possam re...,PSD's proposal provides that tenants can redee...
7,"Durante duas semanas de confinamento, um grupo...","During two weeks of confinement, a group of re..."
8,No final das votações vai saber-se qual o part...,"At the end of the polls, it will be known whic..."
9,Propostas do BE e PCP foram rejeitadas com vot...,BE and PCP proposals were rejected with votes ...


## Classification

In [155]:
# Allocate a pipeline for sentiment-analysis
nlp = pipeline('sentiment-analysis')

def getSentiment(sentence):
    sentiment = nlp(sentence)[0]['label']
    return sentiment

def getScore(sentence):
    score = nlp(sentence)[0]['score']
    return score

In [156]:
df_news['descricao_classification'] = df_news['descricao_en'].apply(getSentiment)
df_news['descricao_score'] = df_news['descricao_en'].apply(getScore)

In [157]:
df_news.head()

,data,titulo,descricao,source,titulo_clean,titulo_en,classification,score,descricao_clean,descricao_en,descricao_classification,descricao_score
0,2020-06-30,Ex-ministro da Saúde Correia de Campos alerta ...,"“Preparemo-nos para o que aí vem”, avisou o an...",publico,Ex-ministro da Saúde Correia de Campos alerta ...,Former Health Minister Correia de Campos warns...,NEGATIVE,0.947701,"“ Preparemo-nos para o que aí vem ” , avisou o...","“Let us prepare for what is coming”, warned th...",NEGATIVE,0.937429
1,2020-06-30,Estados Unidos compram quase todo o <i>stock</...,A compra das mais de 500 mil doses do medicame...,publico,Estados Unidos compram quase todo o stock de r...,United States buys almost all stock of remdesivir,NEGATIVE,0.852459,A compra das mais de 500 mil doses do medicame...,The purchase of more than 500 thousand doses o...,POSITIVE,0.849041
2,2020-06-30,Governo admite quarentena para hotéis com cont...,Hóspedes e funcionários também poderão ficar d...,publico,Governo admite quarentena para hotéis com cont...,Government admits quarantine for hotels with c...,NEGATIVE,0.983680,Hóspedes e funcionários também poderão ficar d...,Guests and staff may also be quarantined in th...,NEGATIVE,0.960382
3,2020-06-30,Covid-19: quem quer ajudar os cientistas a seg...,A perda de olfacto é um dos sintomas comuns as...,publico,Covid-19 : quem quer ajudar os cientistas a se...,Covid-19: who wants to help scientists track t...,NEGATIVE,0.998983,A perda de olfacto é um dos sintomas comuns as...,Loss of smell is one of the common symptoms as...,NEGATIVE,0.992243
4,2020-06-30,"Profissionais do SNS “não deram o litro, deram...","O documento foi apresentado no Infarmed, em Li...",publico,"Profissionais do SNS “ não deram o litro , der...","SNS professionals “did not give the liter, the...",NEGATIVE,0.962007,"O documento foi apresentado no Infarmed , em L...","The document was presented at Infarmed, in Lis...",NEGATIVE,0.998898


In [101]:
df_news['classification'].value_counts()

NEGATIVE    27360
POSITIVE    10233
Name: classification, dtype: int64

In [158]:
df_news['descricao_classification'].value_counts()

NEGATIVE    28239
POSITIVE     9354
Name: descricao_classification, dtype: int64

In [159]:
df_news.to_csv('noticias_en_classification_2.csv')